In [2]:
import pickle as plk
import pandas as pd
from datasets import load_from_disk

ds = load_from_disk('/mnt/pi/datasets/speech/yt_dataset')
ds_len = len(ds['train'])

f= '/home/kd/Desktop/proj/apr/Punjabi_ASR/alignment_probs_for_yt_dataset.pkl'

with open(f, 'rb') as f:
    results = plk.load(f)

result_len = len(results)

print(f'Total ds: {ds_len}, Total results: {result_len}; Diff in %age = {((ds_len - result_len) / ds_len) * 100}')

Loading dataset from disk:   0%|          | 0/400 [00:00<?, ?it/s]

Total ds: 272838, Total results: 272838; Diff in %age = 0.0


In [ ]:
ds['train'][0]

In [6]:
results[13]['text'] == ds['train'][results[13]['index']]['text']

True

In [ ]:
from tqdm import tqdm
import copy
from collections import Counter
MIN_WORD_PROB_THRESHOLD = 0.3

last_word_edge_case_count = 0
for example in results:
    if example['all_ds'] == None:
        continue
    avg_words_map = []
    example['avg'] = copy.deepcopy(example['all_ds'])
    if 'words_map' not in example['indicvoice_verbatim']:
        example['indicvoice_verbatim']['words_map'] = []
        example['all_ds']['words_map'] = []
    try:
        len_words = len(example['all_ds']['words_map'])
        for idx, ((wx, _, px), (wy, _, py)) in enumerate(zip(example['indicvoice_verbatim']['words_map'], example['all_ds']['words_map'])):
            
            if idx == len_words - 2 and px < .10 and py > MIN_WORD_PROB_THRESHOLD:
                    avg_words_map.append((wx, 'avg', py/2))
                    last_word_edge_case_count += 1
            # manual analysis weightage
            elif py < MIN_WORD_PROB_THRESHOLD:
                avg_words_map.append((wx, 'avg', py + (px/2)))
            else:
                avg_words_map.append((wx, 'avg', py))
    except:
        print(f'Error in example: {example["index"]}')
        raise Exception('Error')
    example['avg']['words_map'] = avg_words_map

print(f'last_word_edge_case_count: {last_word_edge_case_count}')
def get_ins_word(words_map):
    assert(len(words_map) > 0)
    min_word, min_prob = words_map[0]
    for (w, p) in words_map:
        if p < min_prob:
            min_word, min_prob = w, p
    return min_word, min_prob


from collections import Counter
exceptions = ['ਹੈ', 'ਅ', 'ਆ', 'ਐ', 'ਹੋ','ਹੀ', 'ਹਾਂ','ਸੀ', 'ਵਿੱਚ', 'ਏ', 'ਈ', 'ਓਂ']
def get_min_words_and_probs_within_threshold(results_filtered, key, threshold):
    grouped_data = {}

    # Initialize grouping structure with counters for each source
    for res in results_filtered:
        source = res['source']
        if source not in grouped_data:
            grouped_data[source] = {
                'data': [],
                'indexes': [],  # Indexes of examples for this source
                'total_entries': 0  # Counter for total entries per source
            }
        
        grouped_data[source]['total_entries'] += 1  # Increment total entries for this source

        text_len = len(res['text'])
        min_word, min_word_prob = None, 1
        
        for (w, _, p), (w2, _, p2) in zip(res[key]['words_map'], res['avg']['words_map']):
            # special cases observed
            if w in exceptions:
                continue
            if p < threshold and p < min_word_prob and p2 < threshold:
                min_word_prob = p
                min_word = w

        if min_word_prob < 1:
            grouped_data[source]['data'].append((min_word, min_word_prob))
            grouped_data[source]['indexes'].append(res['index'])

    # Process and print data for each source
    for source, details in grouped_data.items():
        data = details['data']
        total_entries = details['total_entries']

        insertions = (len(data) / total_entries) * 100 if total_entries else 0
        word_lengths = [len(w) for w, p in data]
        word_lengths_percentages = {l: (word_lengths.count(l) / len(word_lengths)) * 100 for l in set(word_lengths)}
        
        print(f'Source: {source}')
        print(f'{key} Insertions: {insertions:.2f}%')
        print('Min word length distribution:')
        print(Counter(word_lengths))
        print('Min word length percentage distribution:')
        for length, percentage in word_lengths_percentages.items():
            print(f"Length {length}: {percentage:.2f}%")
        print('\n------------------------------------------------------------------\n')

    return grouped_data

def get_words_and_probs_all_above_threshold(results_filtered, key, threshold):
    grouped_data = {}

    # Initialize grouping structure with counters for each source
    for res in results_filtered:
        source = res['source']
        if source not in grouped_data:
            grouped_data[source] = {
                'data': [],
                'indexes': [],  # Indexes of examples for this source
                'total_entries': 0  # Counter for total entries per source
            }
        
        grouped_data[source]['total_entries'] += 1  # Increment total entries for this source
        words_above_threshold = []

        # Check all words to ensure each has a probability above the threshold
        all_above = True
        for w, _, p in res[key]['words_map']:
            if p <= threshold:
                all_above = False
                break
            words_above_threshold.append((w, p))

        # If all words are above threshold, add the data to the group
        if all_above:
            grouped_data[source]['data'].extend(words_above_threshold)
            grouped_data[source]['indexes'].append(res['index'])

    # Process and print data for each source
    for source, details in grouped_data.items():
        data = details['data']
        total_entries = details['total_entries']

        insertions = (len(data) / total_entries) * 100 if total_entries else 0
        word_lengths = [len(w) for w, p in data]
        word_lengths_percentages = {l: (word_lengths.count(l) / len(word_lengths)) * 100 for l in set(word_lengths)}
        
        print(f'Source: {source}')
        print(f'{key} Insertions: {insertions:.2f}%')
        print('Word length distribution:')
        print(Counter(word_lengths))
        print('Word length percentage distribution:')
        for length, percentage in word_lengths_percentages.items():
            print(f"Length {length}: {percentage:.2f}%")
        print('\n------------------------------------------------------------------\n')

    return grouped_data


count_delx, count_dely, count_delavg = 0, 0, 0
verbatim_deletion_indexes = []
samples_removed_due_to_numeric_or_different_lang = 0

for example in tqdm(results):
    if example['all_ds'] == None:
        samples_removed_due_to_numeric_or_different_lang += 1
        continue
    if example['source'] == None:
        example['source'] = 'indicsuperb' # forgot this :;
    delinx, deliny = False, False
    if len(example['indicvoice_verbatim']['has_deleted_word']) > 0:
        count_delx += 1
        delinx = True
        verbatim_deletion_indexes.append(example['index'])
    if len(example['all_ds']['has_deleted_word']) > 0:
        count_dely += 1
        deliny = True
    if delinx and deliny:
        count_delavg += 1

results_after_deletion_removal = [e for e in results if e['index'] not in verbatim_deletion_indexes]

print(f'Percentage of samples removed due to numeric or different language: {samples_removed_due_to_numeric_or_different_lang / result_len * 100}')
remaining_total_ds_len = ds_len - samples_removed_due_to_numeric_or_different_lang
print(f'Percentage of examples with deleted words Verbatim: {count_delx / remaining_total_ds_len * 100}')
print(f'Percentage of examples with deleted words Base    : {count_dely / remaining_total_ds_len * 100}')
print(f'Percentage of examples with deleted words Average : {count_delavg / remaining_total_ds_len * 100}')

print('\n\n')

# remove numeric or different language samples
results_after_deletion_removal = [e for e in results if e['all_ds'] != None]

# get_min_words_and_probs_within_threshold(results_after_deletion_removal, 'indicvoice_verbatim', MIN_WORD_PROB_THRESHOLD)
# all_ds_res = get_min_words_and_probs_within_threshold(results_after_deletion_removal, 'all_ds', MIN_WORD_PROB_THRESHOLD)
avg_res = get_min_words_and_probs_within_threshold(results_after_deletion_removal, 'avg', MIN_WORD_PROB_THRESHOLD)

In [8]:
insertion_indexes = []
for source, data in avg_res.items():
    l = data['indexes']
    insertion_indexes.extend(l)

after_insertion = len(results_after_deletion_removal) - len(insertion_indexes)
print(f'Total Dataset len: {len(results)} -> 100%\nAfter number, different languaage and transcript word deletion: {len(results_after_deletion_removal)} -> {len(results_after_deletion_removal)*100/len(results):.2f}\nAfter insertion: {after_insertion} -> {after_insertion*100/len(results):.2f}')

Total Dataset len: 272838 -> 100%
After number, different languaage and transcript word deletion: 245061 -> 89.82
After insertion: 159693 -> 58.53


In [9]:
len(insertion_indexes)

85368

In [11]:
results_after_insertion_removal = [e for e in results_after_deletion_removal if e['index'] not in insertion_indexes]
final_indexes = [e['index'] for e in results_after_insertion_removal]
print(len(results_after_insertion_removal))

# save as json
import json
with open(f'./selected_indexes_yt_dataset_{after_insertion*100/len(results):.2f}_percent.json', 'w') as f:
    json.dump(final_indexes, f)

159693


In [ ]:
ds_sources = []
for example in results:
    ds_sources.append(example['source'])

s = pd.Series(ds_sources)
print(s.value_counts())